In [1]:
import pandas as pd
import numpy as np
import pickle

# EPO - MUP Data 

In [2]:
# Read epo to mup data
df_epovvc = pd.read_csv(r"V:\midp\match\epo2019\epovvc2019.txt"
                        ,sep="\t"
                        ,encoding="utf-8")

In [3]:
df_epovvc.shape

(573424, 10)

In [4]:
df_epovvc.head()

,person_id,crefo,equal,bestcrefo,appln_id,appln_kind,appln_fili,inpadoc_fa,earliest_f,granted
0,20,2150355978,1,1,16996827,A,1993-04-07,11221689,1992-04-07,Y
1,20,2150355978,1,1,17050088,A,1994-02-22,11252014,1993-02-27,Y
2,20,2150355978,1,1,15716114,A,1994-06-16,6554670,1993-06-16,Y
3,20,2150355978,1,1,16018708,A,1994-06-16,6554670,1993-06-16,Y
4,20,2150355978,1,1,16018709,A,1994-06-16,6554670,1993-06-16,Y


In [5]:
# One crefo may have many patents (different appln_id/person_id)
df_epovvc.loc[df_epovvc.crefo==2150355978,:]

,person_id,crefo,equal,bestcrefo,appln_id,appln_kind,appln_fili,inpadoc_fa,earliest_f,granted
0,20,2150355978,1,1,16996827,A,1993-04-07,11221689,1992-04-07,Y
1,20,2150355978,1,1,17050088,A,1994-02-22,11252014,1993-02-27,Y
2,20,2150355978,1,1,15716114,A,1994-06-16,6554670,1993-06-16,Y
3,20,2150355978,1,1,16018708,A,1994-06-16,6554670,1993-06-16,Y
4,20,2150355978,1,1,16018709,A,1994-06-16,6554670,1993-06-16,Y
...,...,...,...,...,...,...,...,...,...,...
498599,4188845,2150355978,1,1,336078805,A,2011-06-22,335355853,2010-07-02,N
514719,4874703,2150355978,1,1,331580332,A,2004-04-15,16142256,2003-04-15,Y
516272,4971295,2150355978,1,1,334678245,A,2011-05-10,334200018,2010-05-28,N
516273,4971295,2150355978,1,1,336534206,A,2011-07-11,336021830,2010-07-26,Y


In [6]:
# There are only 1s for bestcrefo. No need to drop rows
df_epovvc.bestcrefo.value_counts(dropna=False)

1    573424
Name: bestcrefo, dtype: int64

In [7]:
# Trainings data comprises 36868 firms with known patent match
len(pd.unique(df_epovvc.crefo))

36868

In [8]:
df_tech = pd.read_csv(r"V:\midp\match\epo2019\tech.txt"
                     ,sep="\t"
                     ,encoding="utf-8")

In [9]:
# One application ID may be assigned to several technology classes
df_tech.head()

,appln_id,technology
0,658,17
1,660,17
2,660,20
3,662,32
4,664,20


In [10]:
df_tech.shape

(834270, 2)

In [11]:
# There are 35 distinct technology classes
len(pd.unique(df_tech.technology))

35

In [15]:
df_ipc = pd.read_csv(r"V:\midp\match\epo2019\ipc.txt"
                     ,sep="\t"
                     ,encoding="utf-8")

In [19]:
df_ipc.head()

,appln_id,ipc
0,57,C23C 26/02
1,57,C23C 30/00
2,58,G06T 7/00
3,59,B23B 31/02
4,59,B23B 31/117


In [20]:
df_ipc.shape

(1966462, 2)

In [21]:
# There are 59915 different ipc classes
len(pd.unique(df_ipc.ipc))

59915

In [24]:
# There are 123 different ipc classes (3-digit level)
len(pd.unique(df_ipc.ipc.apply(lambda x: x[0:3])))

123

# Merge Data 

In [25]:
# Merge both datasets
df = df_epovvc.merge(df_tech, how = "left").astype({"technology": "object"}).merge(df_ipc, how = "left")

In [26]:
df.head()

,person_id,crefo,equal,bestcrefo,appln_id,appln_kind,appln_fili,inpadoc_fa,earliest_f,granted,technology,ipc
0,20,2150355978,1,1,16996827,A,1993-04-07,11221689,1992-04-07,Y,2,H04J 3/00
1,20,2150355978,1,1,16996827,A,1993-04-07,11221689,1992-04-07,Y,2,H04J 3/06
2,20,2150355978,1,1,16996827,A,1993-04-07,11221689,1992-04-07,Y,2,H04L 5/22
3,20,2150355978,1,1,16996827,A,1993-04-07,11221689,1992-04-07,Y,2,H04M 11/00
4,20,2150355978,1,1,16996827,A,1993-04-07,11221689,1992-04-07,Y,2,H04N 5/44


In [27]:
df.shape

(3708119, 12)

In [28]:
# Write data
with open(r'Q:\Meine Bibliotheken\Research\Projects\Ausgründungen\02_Data\02_Webdata2IPC\df_epovvc_s0.pkl', 'wb') as f:
    pickle.dump(obj=df, file=f)

# Crefo & URL for Scraping 

In [70]:
df_url = pd.read_csv(r"I:\!Projekte\BMBF_TOBI_131308\01_Arbeitspakete\01_Webscraper\Webscraper\URLs\2019_lebende\MUP_2019_URLs_raw.txt"
                    ,sep="\t"
                    ,encoding="latin-1"
                    ,header=None
                    ,names=['crefo', 'url'] )

In [71]:
# Strip whitespaces in url column
df_url["url"] = df_url.url.apply(lambda x: x.strip())

In [67]:
# Merge with crefos
df_crefo = pd.DataFrame(pd.unique(df.crefo), columns={"crefo"}).merge(df_url, how = "left")

,crefo,url
0,2150355978,www.ip.philips.com
1,5330210848,www.saltigo.de
2,5330000056,www.bayer.com
3,2010636962,www.aca-berlin.de
4,3030129533,www.amtec-chemnitz.de


In [83]:
# Filter only crefos with url
df_crefo = df_crefo.loc[df_crefo.url.notnull(),:]

In [85]:
# Write data
df_crefo.to_csv(r"Q:\Meine Bibliotheken\Research\Projects\Ausgründungen\02_Data\02_Webdata2IPC\urls.txt"
               ,sep = "\t"
               ,encoding="utf-8"
               ,index=False)